In [57]:
import pandas as pd
import numpy as np
import os

from tmdbv3api import Movie
from tmdbv3api import TMDb
import requests

In [73]:
#for i in range(2018,2024):
def get_movies(year):
    link = f"https://en.wikipedia.org/wiki/List_of_American_films_of_{year}"
    df1 = pd.read_html(link, header=0)[2]
    df2 = pd.read_html(link, header=0)[3]
    df3 = pd.read_html(link, header=0)[4]
    df4 = pd.read_html(link, header=0)[5]
    df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

    return df


In [69]:
df_2017 = get_movies(2017)
df_2018 = get_movies(2018)
df_2019 = get_movies(2019)
df_2020 = get_movies(2020)
df_2021 = get_movies(2021)
df_2022 = get_movies(2022)
df_2023 = get_movies(2023)



In [77]:
df_2017 = df_2017[["Title", "Cast and crew"]]
df_2018 = df_2018[["Title", "Cast and crew"]]
df_2019 = df_2019[["Title", "Cast and crew"]]
df_2020 = df_2020[["Title", "Cast and crew"]]
df_2021 = df_2021[["Title", "Cast and crew"]]
df_2022 = df_2022[["Title", "Cast and crew"]]
df_2023 = df_2023[["Title", "Cast and crew"]]


In [144]:
def get_genres(movie_name):
    tmdb = TMDb()
    tmdb_movie = Movie()

    genres = " "
    result = tmdb_movie.search(movie_name)

    if len(result) > 0 :
        id = result[0]["id"]
    else:
        id = np.NaN

    url = f"http://api.themoviedb.org/3/movie/{id}?api_key={os.environ.get('api_key')}"
    response = requests.get(url)
    data = response.json()
    
    if len(data) > 3:
        for i in range(0,len(data.get("genres"))):
             genres +=((data["genres"][i]["name"])) 
        return genres
    else:
        return np.NaN
    


In [177]:
df_2021["Title"] = df_2021["Title"].astype("str") 
df_2022["Title"] = df_2022["Title"].astype("str") 
df_2023["Title"] = df_2023["Title"].astype("str") 

In [178]:

df_2017["genres"] = df_2017["Title"].apply(get_genres)
df_2018["genres"] = df_2018["Title"].apply(get_genres)
df_2019["genres"] = df_2019["Title"].apply(get_genres)
df_2020["genres"] = df_2020["Title"].apply(get_genres)
df_2021["genres"] = df_2021["Title"].apply(get_genres)
df_2022["genres"] = df_2022["Title"].apply(get_genres)
df_2023["genres"] = df_2023["Title"].apply(get_genres)

In [190]:
df_total = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index=True)

In [191]:
df_total

,Title,Cast and crew,genres
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...,FantasyActionThrillerHorror
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...,ThrillerCrime
2,Between Us,Rafael Palacio Illingworth (director/screenpla...,Drama
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...,ActionComedyScience Fiction
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...,HorrorThriller
...,...,...,...
1934,Drive-Away Dolls,Ethan Coen (director/screenplay); Tricia Cooke...,CrimeComedy
1935,It Lives Inside,Bishal Dutta (director/screenplay); Megan Suri...,Horror
1936,PAW Patrol: The Mighty Movie,"Cal Brunker (director); Mckenna Grace, Taraji ...",AnimationFamilyComedyAction
1937,The Creator,Gareth Edwards (director/screenplay); Chris We...,Science FictionActionThriller


In [197]:
df_total["Cast and crew"][0]

'Anna Foerster (director); Cory Goodman (screenplay); Kate Beckinsale, Theo James, Lara Pulver, James Faulkner, Charles Dance'

In [218]:
def get_director(x):
    return str(x).split("(")[0]

df_total["director"] = df_total["Cast and crew"].apply(get_director)


In [223]:
df_total["Cast and crew"][0]

'Anna Foerster (director); Cory Goodman (screenplay); Kate Beckinsale, Theo James, Lara Pulver, James Faulkner, Charles Dance'

In [219]:
df_total.head(30)

,Title,Cast and crew,genres,director
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...,FantasyActionThrillerHorror,Anna Foerster
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...,ThrillerCrime,Steven C. Miller
2,Between Us,Rafael Palacio Illingworth (director/screenpla...,Drama,Rafael Palacio Illingworth
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...,ActionComedyScience Fiction,Chris Wedge
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...,HorrorThriller,Stacy Title
5,Sleepless,Baran bo Odar (director); Andrea Berloff (scre...,ActionCrimeThriller,Baran bo Odar
6,100 Streets,Jim O'Hanlon (director); Leon F. Butler (scree...,Drama,Jim O'Hanlon
7,The Book of Love,Bill Purple (director/screenplay); Robbie Pick...,Drama,Bill Purple
8,Split,M. Night Shyamalan (director/screenplay); Jame...,HorrorThriller,M. Night Shyamalan
9,XXX: Return of Xander Cage,D. J. Caruso (director); F. Scott Frazier (scr...,ActionAdventureCrime,D. J. Caruso
